In [1]:
import tensorrt as trt
import uff
from tensorrt.parsers import uffparser

## Settings

In [2]:
FROZEN_FPATH = 'data/frozen.pb' # ADJUST
ENGINE_FPATH = 'data/engine.plan' # ADJUST
INPUTE_NODE = 'net/input' # ADJUST
ONPUT_NODE = 'net/fc8/BiasAdd' # ADJUST
INPUT_SIZE = [3, 224, 224] # ADJUST
MAX_BATCH_SIZE = 1 # ADJUST
MAX_WORKSPACE = 1 << 20 # ADJUST

## Convert TF frozen graph to UFF graph

In [3]:
uff_model = uff.from_tensorflow_frozen_model(FROZEN_FPATH, [ONPUT_NODE])

Using output node net/fc8/BiasAdd
Converting to UFF graph
No. nodes: 95


## Create UFF parser and logger

In [4]:
parser = uffparser.create_uff_parser()
parser.register_input(INPUTE_NODE, INPUT_SIZE, 0)
parser.register_output(ONPUT_NODE)
G_LOGGER = trt.infer.ConsoleLogger(trt.infer.LogSeverity.INFO)

## Build optimized inference engine

In [5]:
engine = trt.utils.uff_to_trt_engine(
    G_LOGGER, uff_model, parser, MAX_BATCH_SIZE, MAX_WORKSPACE)

## Save inference engine

In [6]:
trt.utils.write_engine_to_file(ENGINE_FPATH, engine.serialize())

True

## Cleaning Up 

In [7]:
parser.destroy()
engine.destroy()